In [2]:
import os, time, gc
import numpy as np
import datatable as dt
import pandas as pd
import xgboost as xgb
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from numba import njit
import treelite, treelite_runtime
import optuna

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn import set_config
set_config(display='diagram')

import warnings
warnings.filterwarnings('ignore')

In [2]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [3]:
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [4]:
%%time
path = '/kaggle/working/input/'
# path = '/kaggle/input/jane-street-market-prediction'
# train = dt.fread(os.path.join(path, 'train.csv')).to_pandas()
# train = train[train['weight'] != 0].reset_index(drop = True)
train_data = (
    dt.fread(os.path.join(path, 'train.csv'))
    .to_pandas()
    .query('weight > 0')
    .reset_index(drop=True)
)
f_mean = train_data.mean()
train_data = train_data.fillna(f_mean)

feature_names = [c for c in train_data.columns if 'feature' in c]

train_data['action'] = (train_data['resp'].values > 0).astype(int)

CPU times: user 13min 32s, sys: 33 s, total: 14min 5s
Wall time: 25.8 s


In [5]:
X_train = train_data[feature_names].values
X_weight = train_data['weight'].values
X_date = train_data['date'].values
X_resp = train_data['resp'].values
y_labels = train_data['action'].values
groups = train_data['date'].values

In [6]:
cv = PurgedGroupTimeSeriesSplit(
    n_splits=3,
    max_train_group_size=150,
    max_test_group_size=60,
    group_gap=20
)

def objectie(trial, cv=cv, cv_fold_func=np.average):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 350, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 0.90),
        'gamma': trial.suggest_int('gamma', 0, 20),
        'missing': -999,
        'tree_method': 'gpu_hist' 
    }
    aucs = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(
        X_train,
        y_labels,
        groups
    )):
        
        d_train = xgb.DMatrix(X_train[train_idx], y_labels[train_idx])
        d_valid_x = xgb.DMatrix(X_train[valid_idx])
        clf = xgb.train(params=params, dtrain=d_train, num_boost_round=200)
        preds = clf.predict(d_valid_x)
        auc = utility_score_bincount(
            action=preds, 
            weight=X_weight[valid_idx], 
            resp=X_resp[valid_idx],
            date=X_date[valid_idx],
        )
        aucs.append(auc)
        
    print(f'Trial done: AUC values on folds: {aucs}')
    return cv_fold_func(aucs)
gc.collect()

44

In [7]:
FIT_XGB = True
n_trials = 60

if FIT_XGB:
    study = optuna.create_study(direction='maximize')
    study.optimize(objectie, n_trials)
    
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("Value: {}".format(trial.value))

    print("Params:")
    for key, value in trial.params.items():
        print("{}: {}".format(key, value))

[I 2021-02-13 14:32:11,467] A new study created in memory with name: no-name-ecba5849-d16c-471b-bdbd-aa578d892538


[14:32:15] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:22] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:29] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

[I 2021-02-13 14:32:32,530] Trial 0 finished with value: 0.29875175656710506 and parameters: {'n_estimators': 451, 'max_depth': 8, 'learning_rate': 0.07032854217178496, 'subsample': 0.5111717881380816, 'colsample_bytree': 0.5353314591052204, 'gamma': 13}. Best is trial 0 with value: 0.29875175656710506.


Trial done: AUC values on folds: [-0.0, -0.0, 0.8962552697013152]
[14:32:37] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:44] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:52] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not us

[I 2021-02-13 14:32:56,005] Trial 1 finished with value: 0.4305193332024578 and parameters: {'n_estimators': 371, 'max_depth': 6, 'learning_rate': 0.029902334245203742, 'subsample': 0.5975051391923565, 'colsample_bytree': 0.5450317291663184, 'gamma': 12}. Best is trial 1 with value: 0.4305193332024578.


...

[I 2021-02-13 14:40:53,617] Trial 18 finished with value: 8.812638805326936 and parameters: {'n_estimators': 557, 'max_depth': 9, 'learning_rate': 0.07681403599153656, 'subsample': 0.7837055658731438, 'colsample_bytree': 0.7982550134121199, 'gamma': 5}. Best is trial 13 with value: 32.525189727572304.


Trial done: AUC values on folds: [0.11001497866389837, -0.0, 26.327901437316907]
[14:40:57] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:41:09] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:41:23] WARNING: ../src/learner.cc:541: 
Parameters: { missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parame

[I 2021-02-13 14:41:32,934] Trial 19 finished with value: 20.468317211012916 and parameters: {'n_estimators': 411, 'max_depth': 7, 'learning_rate': 0.04259795777736618, 'subsample': 0.6810182845845397, 'colsample_bytree': 0.719607121135183, 'gamma': 2}. Best is trial 13 with value: 32.525189727572304.


Trial done: AUC values on folds: [3.2207547938168206, -0.0, 58.184196839221926]


KeyboardInterrupt: 

In [ ]:
best_params = trial.params
best_params['tree_method'] = 'gpu_hist' 
best_params
with open('./params.txt', "w", encoding='utf-8') as f:
    f.write(str(best_params))
    f.close()

In [ ]:
%%time
dtrain = xgb.DMatrix(X_train, y_labels)
clf = xgb.train(best_params, dtrain, 200)

## Treelite

In [ ]:
model = treelite.Model.from_xgboost(clf)
toolchain = 'gcc'
model.export_lib(toolchain=toolchain, libpath='./mymodel.so', params={'parallel_comp': 32}, verbose=True)

In [1]:
predictor = treelite_runtime.Predictor('./mymodel.so', verbose=True)

NameError: name 'treelite_runtime' is not defined

In [ ]:
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [ ]:
%%time
PCA_appied = False
opt_th = 0.505
tmp = np.zeros(len(feature_names))
for (test_df, prediction_df) in tqdm(iter_test):
    if test_df['weight'].values[0] > 0:
        x_tt = test_df.loc[:, feature_names].values
        x_tt[0, :] = fast_fillna(x_tt[0, :], tmp)
        tmp = x_tt[0, :]
        if PCA_appied:
#             y_preds = clf.predict(pca.transform(x_tt))
            y_preds = predictor.predict(treelite_runtime.DMatrix(pca.transform(x_tt)))
        else:
#             y_preds = clf.predict(x_tt)
            y_preds = predictor.predict(treelite_runtime.DMatrix(x_tt))
        prediction_df["action"].values[0] = int(y_preds >= 0.5)
    else:
        prediction_df["action"].values[0] = 0
    env.predict(prediction_df)